In [39]:
import pandas as pd
house_data=pd.read_csv('cal_housing_clean.csv')
house_data.head()


,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [40]:
house_data.describe().transpose()


,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [41]:
X_data=house_data.drop('medianHouseValue',axis=1)
X_data.head()


,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
0,41.0,880.0,129.0,322.0,126.0,8.3252
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014
2,52.0,1467.0,190.0,496.0,177.0,7.2574
3,52.0,1274.0,235.0,558.0,219.0,5.6431
4,52.0,1627.0,280.0,565.0,259.0,3.8462


In [42]:
Y_data=house_data['medianHouseValue']
Y_data.head()


0    452600.0
1    358500.0
2    352100.0
3    341300.0
4    342200.0
Name: medianHouseValue, dtype: float64

In [0]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X_data,Y_data,test_size=0.30,random_state=101)


In [44]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(X_train)


MinMaxScaler(copy=True, feature_range=(0, 1))

In [0]:
X_train=pd.DataFrame(data=scaler.transform(X_train),
                    columns=X_train.columns,
                    index=X_train.index)

X_test=pd.DataFrame(data=scaler.transform(X_test),
                    columns=X_test.columns,
                    index=X_test.index)

In [46]:
#Create Feature Columns
house_data.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [0]:
import tensorflow as tf

house_age=tf.feature_column.numeric_column('housingMedianAge')
house_room=tf.feature_column.numeric_column('totalRooms')
house_bedroom=tf.feature_column.numeric_column('totalBedrooms')
population=tf.feature_column.numeric_column('population')
households=tf.feature_column.numeric_column('households')
medianIncome=tf.feature_column.numeric_column('medianIncome')

feat_cols=[house_age,house_room,house_bedroom,population,households,medianIncome]



In [0]:
#Create input function

input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,
                                               y=Y_train,batch_size=10,num_epochs=1000,shuffle=True)

In [49]:
#Create Model

model=tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpupqca965', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f753c002978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [50]:
#train the model
model.train(input_fn=input_func,steps=20000)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpupqca965/model.ckpt.
INFO:tensorflow:loss = 256085260000.0, step = 1
INFO:tensorflow:global_step/sec: 451.196
INFO:tensorflow:loss = 567602450000.0, step = 101 (0.227 sec)
INFO:tensorflow:global_step/sec: 463.783
INFO:tensorflow:loss = 543007740000.0, step = 201 (0.215 sec)
INFO:tensorflow:global_step/sec: 469.276
INFO:tensorflow:loss = 460591730000.0, step = 301 (0.216 sec)
INFO:tensorflow:global_step/sec: 454.072
INFO:tensorflow:loss = 653417000000.0, step = 401 (0.216 sec)
INFO:tensorflow:global_step/sec: 467.491
INFO:tensorflow:loss = 481723300000.0, step = 501 (0.215 sec)
INFO:tensorflow:global_step/sec: 458.483
INFO:tensorflow:loss = 324771000000.0, step = 601 (0.218 sec)
INFO:tensorflow:global

In [51]:
#create predict input function
predict_input_fun=tf.estimator.inputs.pandas_input_fn(x=X_test,
                                                     batch_size=10,
                                                     num_epochs=1,
                                                     shuffle=False)
pred_gen=model.predict(predict_input_fun)

predictions=list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpupqca965/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [0]:
final_preds=[]
for pred in predictions:
  final_preds.append(pred['predictions'])

In [53]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test,final_preds)**0.5


94378.10380709176